Step 1: Install Required Packages

In [ ]:
# Install necessary libraries
%pip install --upgrade --quiet langchain langchain-community langchain-groq neo4j python-dotenv

Step 2: Import Libraries

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# Print to check if variables are loaded (for debugging)
print(f"GROQ_API_KEY: {GROQ_API_KEY}")
print(f"NEO4J_URI: {NEO4J_URI}")
print(f"NEO4J_USERNAME: {NEO4J_USERNAME}")


Step 3: Connect to Neo4j and Import Data

In [ ]:
from langchain_community.graphs import Neo4jGraph

# Create a graph connection using environment variables
graph = Neo4jGraph(uri=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

# Define the Cypher query to load movie data
movies_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

# Execute the query to populate the graph
graph.query(movies_query)


Step 4: Define the Graph Schema

In [ ]:
graph.refresh_schema()
print(graph.schema)


Step 5: Set Up the Q&A Chain with Groq

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from langchain.chains import GraphCypherQAChain

# Initialize the HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize the Groq LLM
groq_model = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

# Create the GraphCypherQAChain with Groq
chain = GraphCypherQAChain.from_llm(graph=graph, llm=groq_model, verbose=True, embed_model=embed_model)

# Define a query and get the response
response = chain.invoke({"query": "What was the cast of the Casino?"})
print(response)


Step 6: Validate Relationship Directions 

In [ ]:
chain = GraphCypherQAChain.from_llm(
    graph=graph, llm=groq_model, verbose=True, validate_cypher=True, embed_model=embed_model
)

# Define a query and get the response
response = chain.invoke({"query": "What was the cast of the Casino?"})
print(response)
